In [ ]:
import utils
import plotting
import postprocessing
import corrections

from utils import CUT_MAX_VAL
from hh_vars import (
    years,
    data_key,
    qcd_key,
    bg_keys,
    samples,
    nonres_sig_keys,
    # res_samples,
    # res_sig_keys,
    nonres_samples,
    txbb_wps,
    jec_shifts,
    jmsr_shifts,
)
from postprocessing import res_shape_vars

from collections import OrderedDict

import numpy as np
import pandas as pd
import pickle, json
from pandas.errors import SettingWithCopyWarning
from hist import Hist

import os
from copy import deepcopy
from inspect import cleandoc
import warnings

# ignore these because they don't seem to apply
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
import numpy as np


# low mass list
low_m_higgs = np.array(
    [
        15,
        20,
        25,
        30,
        40,
        50,
        60,
        70,
        80,
        90,
        100,
        110,
        120,
        130,
        140,
        150,
        160,
        170,
        180,
        190,
        200,
        210,
        220,
        230,
        240,
        250,
    ]
)
low_m_res = np.arange(600, 6000, 100)

# high mass list
m_higgs = np.arange(260, 660, 10)
# # minimum m_res changes s.t. it is always > 2x m_higgs to avoid off-shell Higgses
# m_res_min = np.linspace(600, 1600, len(m_higgs))

# reweight points such that there are the same number of events at 260 as 250 GeV
# and then continuously decrease the weight from there till 650 GeV
num_low_points = len(low_m_higgs)
num_high_points = len(m_higgs)

# solve system of equations s.t. 1) total weight sums to 1, and 2) the first weight is 1 / (# of low points) i.e. same # of events as 260 GeV
m = np.array([[num_high_points, num_high_points * (num_high_points - 1) / 2], [1, num_high_points]])
b = np.array([1, 1 / num_low_points])
# a is smallest weight, d is spacing between weights
a, d = np.linalg.inv(m).dot(b)


def mh_weight(mh):
    idx = np.where(m_higgs == mh)[0][0]
    return a + d * (len(m_higgs) - idx - 1)


def mres_min(mh):
    """Choose mX for mH s.t. mX^2 - 4mH^2 remains constant for each mH"""
    mdel = 600**2 - 4 * 250**2
    return np.sqrt(mdel + 4 * mh**2)

In [ ]:
for mh in m_higgs:
    m_res = np.linspace(mres_min(mh), mres_min(mh) * 10, len(low_m_res), endpoint=False)
    for mx in m_res:
        print("BulkGravitonToHH_MX%.0f_MH%.0f weight %.5f" % (mx, mh, mh_weight(mh)))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
res_samples = OrderedDict()

res_mps = [
    (1000, 125),
    (1400, 125),
    (1400, 150),
    (1800, 125),
    (1800, 150),
    (1800, 190),
    (2200, 125),
    (2200, 150),
    (2200, 190),
    (2200, 250),
    (3000, 125),
    (3000, 150),
    (3000, 190),
    (3000, 250),
    (3000, 350),
]

for mX, mY in res_mps:
    res_samples[
        f"X[{mX}]->H(bb)Y[{mY}](VV)"
    ] = f"NMSSM_XToYH_MX{mX}_MY{mY}_HTo2bYTo2W_hadronicDecay"

res_sig_keys = list(res_samples.keys())

In [ ]:
del nonres_samples["VBFHHbbVV"]
samples = nonres_samples | samples

In [ ]:
sig_split_points = [
    [
        (1000, 125),
        (1400, 125),
        (1800, 125),
        (2200, 125),
        (3000, 125),
    ],
    [
        (1400, 150),
        (1800, 150),
        (1800, 190),
        (2200, 150),
        (2200, 190),
        (2200, 250),
    ],
    [
        (3000, 125),
        (3000, 150),
        (3000, 190),
        (3000, 250),
        (3000, 350),
    ],
]

sig_splits = [
    ["HHbbVV"] + [f"X[{mX}]->H(bb)Y[{mY}](VV)" for (mX, mY) in mps] for mps in sig_split_points
]

In [ ]:
sig_splits = [res_sig_keys]

In [ ]:
MAIN_DIR = "../../../"
samples_dir = f"{MAIN_DIR}/../data/skimmer/Feb24"
signal_samples_dir = f"{MAIN_DIR}/../data/skimmer/Mar10_2"
year = "2017"

date = "23May21"
plot_dir = f"../../../plots/PostProcessing/{date}/"
templates_dir = f"templates/{date}/"
for i in range(len(sig_splits)):
    _ = os.system(f"mkdir -p {plot_dir}/ControlPlots/{year}/sigs{i}/")
_ = os.system(f"mkdir -p {plot_dir}/cutflows/")
_ = os.system(f"mkdir -p {plot_dir}/templates/wshifts")
_ = os.system(f"mkdir -p {plot_dir}/templates/jshifts")
_ = os.system(f"mkdir -p {plot_dir}/templates/hists2d")
_ = os.system(f"mkdir -p {templates_dir}")

selection_regions = postprocessing.get_res_selection_regions(year)

Load samples

In [ ]:
# Both Jet's Regressed Mass above 50, electron veto
filters = [
    [
        ("('ak8FatJetParticleNetMass', '0')", ">=", 50),
        ("('ak8FatJetParticleNetMass', '1')", ">=", 50),
        # ("('nGoodElectrons', '0')", "==", 0),
    ],
]
systematics = {year: {}}

# save cutflow as pandas table
cutflow = pd.DataFrame(index=list(samples.keys()) + list(res_samples.keys()))

# utils.remove_empty_parquets(samples_dir, year)
events_dict = utils.load_samples(signal_samples_dir, res_samples, year, filters)
events_dict |= utils.load_samples(samples_dir, samples, year, filters)

utils.add_to_cutflow(events_dict, "Preselection", "weight", cutflow)

print("")
# print weighted sample yields
for sample in events_dict:
    tot_weight = np.sum(events_dict[sample]["weight"].values)
    print(f"Pre-selection {sample} yield: {tot_weight:.2f}")

In [ ]:
higgs_samples = OrderedDict(
    [
        ("Hbb", "*HToBB"),
        # ("HWW", ("*HToWW", "*HToNonbb")),
        # ("HH", ("VBF_HHTobbVV_CV_1_C2V_1_C3_1", "GluGluToHHTo4B_node_cHHH1_preUL")),
    ]
)

events_dict |= utils.load_samples(samples_dir, higgs_samples, year, filters)

cutflow = pd.DataFrame(
    index=list(samples.keys()) + list(res_samples.keys()) + list(higgs_samples.keys())
)
utils.add_to_cutflow(events_dict, "Preselection", "weight", cutflow)

print("")
# print weighted sample yields
for sample in events_dict:
    tot_weight = np.sum(events_dict[sample]["weight"].values)
    print(f"Pre-selection {sample} yield: {tot_weight:.2f}")

In [ ]:
import matplotlib.pyplot as plt

bins = np.arange(-20, 61, 5)
plt.hist(events_dict["ST"]["weight"], bins, histtype="step", label="ST")
plt.hist(events_dict["TT"]["weight"], bins, histtype="step", label="TT")
plt.yscale("log")
plt.ylabel("# Events")
plt.xlabel("Weights")
plt.legend()
plt.savefig(f"{plot_dir}/sttt_weights.pdf")

Scale factors and bb VV assignment

In [ ]:
postprocessing.apply_weights(events_dict, year, cutflow)
bb_masks = postprocessing.bb_VV_assignment(events_dict)
cutflow

In [ ]:
for sample, events in events_dict.items():
    if "ak8FatJetParTMD_THWWvsT" not in events:
        h4qvst = (events["ak8FatJetParTMD_probHWW3q"] + events["ak8FatJetParTMD_probHWW4q"]) / (
            events["ak8FatJetParTMD_probHWW3q"]
            + events["ak8FatJetParTMD_probHWW4q"]
            + events["ak8FatJetParTMD_probQCD"]
            + events["ak8FatJetParTMD_probT"]
        )

        events_dict[sample] = pd.concat(
            [events, pd.concat([h4qvst], axis=1, keys=["ak8FatJetParTMD_THWWvsT"])], axis=1
        )

Control Plots

In [ ]:
# {var: (bins, label)}
control_plot_vars = {
    # "MET_pt": ([50, 0, 300], r"$p^{miss}_T$ (GeV)"),
    # "DijetEta": ([50, -8, 8], r"$\eta^{jj}$"),
    # "DijetPt": ([50, 0, 750], r"$p_T^{jj}$ (GeV)"),
    # "DijetMass": (
    #     # list(range(800, 1400, 100)) + [1400, 1600, 2000, 3000, 4400],
    #     [40, 600, 4500],
    #     r"$m^{jj}$ (GeV)",
    # ),
    # "bbFatJetEta": ([50, -2.4, 2.4], r"$\eta^{bb}$"),
    # "bbFatJetPt": ([50, 300, 1500], r"$p^{bb}_T$ (GeV)"),
    # "bbFatJetParticleNetMass": ([40, 52.5, 252.5], r"$m^{bb}_{reg}$ (GeV)"),
    # "bbFatJetMsd": ([50, 0, 300], r"$m^{bb}_{msd}$ (GeV)"),
    # "bbFatJetParticleNetMD_Txbb": ([50, 0.8, 1], r"$T^{bb}_{Xbb}$"),
    # "VVFatJetEta": ([50, -2.4, 2.4], r"$\eta^{VV}$"),
    # "VVFatJetPt": ([50, 300, 1500], r"$p^{VV}_T$ (GeV)"),
    # "VVFatJetParticleNetMass": (
    #     # list(range(50, 110, 10)) + list(range(110, 200, 15)) + [200, 220, 250],
    #     [20, 50, 250],
    #     r"$m^{VV}_{reg}$ (GeV)",
    # ),
    # "VVFatJetMsd": ([40, 50, 250], r"$m^{VV}_{msd}$ (GeV)"),
    # "VVFatJetParticleNet_Th4q": ([50, 0, 1], r"Prob($H \to 4q$) vs Prob(QCD) (Non-MD)"),
    # "VVFatJetParTMD_THWW4q": (
    #     [50, 0, 1],
    #     r"Prob($H \to VV \to 4q$) vs Prob(QCD) (Mass-Decorrelated)",
    # ),
    # "VVFatJetParTMD_probT": ([50, 0, 1], r"Prob(Top) (Mass-Decorrelated)"),
    # "VVFatJetParTMD_THWWvsT": (
    #     [50, 0, 1],
    #     r"$T^{VV}_{HWW}$",
    # ),
    # "bbFatJetPtOverDijetPt": ([50, 0, 40], r"$p^{bb}_T / p_T^{jj}$"),
    # "VVFatJetPtOverDijetPt": ([50, 0, 40], r"$p^{VV}_T / p_T^{jj}$"),
    # "VVFatJetPtOverbbFatJetPt": ([50, 0.4, 2.0], r"$p^{VV}_T / p^{bb}_T$"),
    "nGoodMuons": ([3, 0, 3], r"# of Muons"),
    "nGoodElectrons": ([3, 0, 3], r"# of Electrons"),
    "nGoodJets": ([5, 0, 5], r"# of AK4 B-Jets"),
}

# hists = postprocessing.control_plots(
#     events_dict,
#     bb_masks,
#     nonres_sig_keys + res_sig_keys,
#     control_plot_vars,
#     f"{plot_dir}/ControlPlots/{year}/",
#     year,
#     sig_splits=sig_splits,
#     # bg_keys=bg_keys + list(higgs_samples.keys()),
#     bg_keys=["QCD", "TT", "ST", "V+Jets", "Hbb"],
#     show=True,
# )

In [ ]:
selection, _ = utils.make_selection(
    {
        "VVFatJetParTMD_THWWvsT": [0.8, CUT_MAX_VAL],
        "bbFatJetParticleNetMD_Txbb": [0.98, CUT_MAX_VAL],
        "bbFatJetParticleNetMass": [110, 145],
    },
    events_dict,
    bb_masks,
)
cutstr = f"pass_noveto"

postprocessing.control_plots(
    events_dict,
    bb_masks,
    nonres_sig_keys + res_sig_keys,
    control_plot_vars,
    f"{plot_dir}/ControlPlots/{year}/",
    year,
    sig_splits=sig_splits[:1],
    hists={},
    # bg_keys=bg_keys + list(higgs_samples.keys()),
    # bg_keys=["QCD", "TT", "ST", "V+Jets", "Hbb"],
    bg_keys=["QCD", "TT", "ST", "V+Jets"],
    sig_scale_dict={key: 10 for key in nonres_sig_keys + res_sig_keys},
    selection=selection,
    cutstr=cutstr,
    show=True,
)

In [ ]:
for sb1, sb2 in [[0, 300], [75, 180], [92.5, 162.5]]:
    selection, _ = utils.make_selection(
        {
            "VVFatJetParTMD_THWWvsT": [0.8, CUT_MAX_VAL],
            "bbFatJetParticleNetMD_Txbb": [0.98, CUT_MAX_VAL],
            "bbFatJetParticleNetMass": [[sb1, 110], [145, sb2]],
        },
        events_dict,
        bb_masks,
    )
    cutstr = f"sidebands_{sb1}_{sb2}"

    postprocessing.control_plots(
        events_dict,
        bb_masks,
        nonres_sig_keys + res_sig_keys,
        control_plot_vars,
        f"{plot_dir}/ControlPlots/{year}/",
        year,
        sig_splits=sig_splits,
        hists={},
        # bg_keys=bg_keys + list(higgs_samples.keys()),
        bg_keys=["QCD", "TT", "ST", "V+Jets", "Hbb"],
        selection=selection,
        cutstr=cutstr,
        show=True,
    )

Overall LP SF

In [ ]:
from collections import OrderedDict
from tqdm import tqdm

sel, cf = utils.make_selection(
    selection_regions["lpsf"].cuts, events_dict, bb_masks, prev_cutflow=cutflow
)

sf_table = OrderedDict()

for sig_key in tqdm(res_sig_keys):
    systematics[sig_key] = {}
    # calculate only for current year
    events_dict[sig_key] = postprocessing.postprocess_lpsfs(events_dict[sig_key])
    lp_sf, unc, uncs = postprocessing.get_lpsf(events_dict[sig_key], sel[sig_key])
    # print(f"BDT LP Scale Factor for {sig_key}: {lp_sf:.2f} ± {unc:.2f}")
    # print(uncs)

    systematics[sig_key]["lp_sf"] = lp_sf
    systematics[sig_key]["lp_sf_unc"] = unc / lp_sf

    sf_table[sig_key] = {"SF": f"{lp_sf:.2f} ± {unc:.2f}", **uncs}

In [ ]:
sf_df = pd.DataFrame(index=nonres_sig_keys + res_sig_keys)

for key in sf_table[sig_key]:
    sf_df[key] = [sf_table[skey][key] for skey in nonres_sig_keys + res_sig_keys]

sf_df.to_clipboard()
sf_df

Templates

In [ ]:
selection_regions = postprocessing.get_res_selection_regions("2017", txbb_wp="HP", thww_wp=0.8)
del selection_regions["fail"], selection_regions["failBlinded"]

In [ ]:
h = postprocessing.get_templates(
    events_dict,
    bb_masks,
    year,
    # nonres_sig_keys + res_sig_keys[:10],
    res_sig_keys,
    selection_regions,
    res_shape_vars[:1],
    systematics,
    templates_dir,
    bg_keys=["QCD", "TT", "V+Jets", "Diboson", "Hbb"],
    plot_dir=f"{plot_dir}/templates/",
    prev_cutflow=cutflow,
    # sig_splits=sig_splits[:2],
    weight_shifts={},
    jshift="",
    plot_shifts=False,
    pass_ylim=70,
    fail_ylim=40000,
    blind_pass=True,
    show=True,
)

In [ ]:
get_templates(
    events_dict,
    bb_masks,
    args.year,
    sig_keys,
    selection_regions,
    shape_vars,
    systematics,
    template_dir,
    bg_keys=bg_keys,
    plot_dir=plot_dir,
    prev_cutflow=cutflow,
    # sig_splits=sig_splits,
    weight_shifts=weight_shifts,
    jshift=jshift,
    blind_pass=True if args.resonant else False,
    show=False,
    plot_shifts=args.plot_shifts,
)

In [ ]:
templates = {}

for jshift in [""] + jec_shifts + jmsr_shifts:
    print(jshift)
    ttemps, tsyst = postprocessing.get_templates(
        events_dict,
        bb_masks,
        year,
        nonres_sig_keys + res_sig_keys,
        res_selection_regions[year],
        res_shape_vars,
        bg_keys=["QCD", "TT", "V+Jets"],
        plot_dir=plot_dir if jshift == "" else "",
        prev_cutflow=cutflow,
        sig_splits=sig_splits,
        weight_shifts=postprocessing.weight_shifts,
        jshift=jshift,
        pass_ylim=7,
        fail_ylim=40000,
        blind_pass=True,
        show=False,
        plot_shifts=False,
    )

    templates = {**templates, **ttemps}
    if jshift == "":
        systematics[year] = tsyst

In [ ]:
with open(f"{templates_dir}/{year}_templates.pkl", "wb") as f:
    pickle.dump(templates, f)

with open(f"{templates_dir}/systematics.json", "w") as f:
    json.dump(systematics, f)

In [ ]:
with open(f"templates/Apr10//2017_templates.pkl", "rb") as f:
    templates = pickle.load(f)

In [ ]:
len(templates["pass"].axes[2])

In [ ]:
plotting.hist2ds(
    templates,
    f"{plot_dir}/templates/hists2d/",
    regions=["pass", "fail", "passBlinded", "failBlinded"],
    region_labels=selection_regions_label,
    samples=["Data", "TT", "V+Jets", "X[3000]->H(bb)Y[190](VV)"],
    # fail_zlim=5e3,
    # pass_zlim=1.0,
)

In [ ]:
systematics

In [ ]:
templates_dict = {}

for year in years:
    with open(f"templates/{date}/{year}_templates.pkl", "rb") as f:
        templates_dict[year] = pickle.load(f)

In [ ]:
templates = []
for year in years:
    with open(f"templates/Apr7//{year}_templates.pkl", "rb") as f:
        templates.append(pickle.load(f))